Video Embeddings with Videoprism

In [1]:
# @title Prepare environment

import os
import sys

# Fetch VideoPrism repository if Python does not know about it and install
# dependencies needed for this notebook.
if not os.path.exists("videoprism_repo"):
  !git clone --quiet --branch=main --depth=1 \
     https://github.com/everettVT/videoprism.git videoprism_repo
  os.chdir('./videoprism_repo')
  !pip install .
  os.chdir('..')

# Append VideoPrism code to Python import path.
if "videoprism_repo" not in sys.path:
  sys.path.append("videoprism_repo")

Processing /content/videoprism_repo
  Preparing metadata (setup.py) ... done
  Created wheel for videoprism: filename=videoprism-1.0.0-py3-none-any.whl size=40354 sha256=befadd51094791299b5172aa934474b2d956e95f94ebfba0c18153b66d6d0570
  Stored in directory: /tmp/pip-ephem-wheel-cache-tz1vq40s/wheels/e3/73/3c/3dc3551ff92b46a1e55f9a893f2d5b8fdc55d670bd73d3b605
Successfully built videoprism


In [2]:
!pip install "daft>=0.6.1" av yt-dlp "jax[cuda12]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 50.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82


In [3]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
The token `Anyscale Ray Serve LLM` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

In [4]:
import daft
from daft import col, DataType as dt
import numpy as np
import jax
import jax.numpy as jnp
from jax.extend import backend
import tensorflow as tf
from videoprism import models as vp

In [5]:
print(jax.devices())    # should list a CUDA device

[CudaDevice(id=0)]
[CudaDevice(id=0)]


- B: batch size (number of videos in a batch).
- T: number of frames per video clip (typically 16).
- N: tokens per frame (for 288×288 with 18×18 patches → 16×16 = 256).
- D: embedding dimension (Base: 768; Large: 1024).

Video-text model returns:
- video_embeddings: [B, D] (global video embeddings).
- text_embeddings: [B, D] (global text embeddings).
- Optional: frame_embeddings [B, T, D]; tokens [B, T×N, D]

Retrieval:
- cosine similarity reduces to dot product since outputs are L2-normalized.
- For a single video vs K texts: [1, D] @ [K, D]^T → [1, K].

In [30]:
128 * 16

2048

In [6]:
PATHS = ["https://www.youtube.com/watch?v=wKOC_w4oKO8"]
B, T, H, W, C = 2, 16, 288, 288, 3
MODEL_NAME = 'videoprism_lvt_public_v1_base' # or 'videoprism_lvt_public_v1_large'

In [31]:
df_frames = daft.read_video_frames(
    PATHS,
    image_height=H,
    image_width=W,
).limit(2048).collect()

🗡️ 🐟 Limit 2048: 00:00 

🗡️ 🐟 PythonFunction Scan: 00:00 

In [9]:
df_frames.show(3)

pathUtf8,frame_indexInt64,frame_timeFloat64,frame_time_baseUtf8,frame_ptsInt64,frame_dtsInt64,frame_durationInt64,is_key_frameBoolean,dataImage[RGB; 288 x 288]
https://www.youtube.com/watch?v=wKOC_w4oKO8,0,0,1/11988,0,0,400,true,""" />"
https://www.youtube.com/watch?v=wKOC_w4oKO8,1,0.0333667000333667,1/11988,400,400,400,false,""" />"
https://www.youtube.com/watch?v=wKOC_w4oKO8,2,0.0667334000667334,1/11988,800,800,400,false,""" />"


### Sampling Strategies

In [32]:
df_grouped = (
    df_frames
    .with_column("group_index", df_frames["frame_index"] // T)
    .groupby("path", "group_index")
    .agg_list("data", "frame_index")
)
df_grouped.show(3)

pathUtf8,group_indexInt64,dataList[Image[RGB; 288 x 288]],frame_indexList[Int64]
https://www.youtube.com/watch?v=wKOC_w4oKO8,22,"[<FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>]","[352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367]"
https://www.youtube.com/watch?v=wKOC_w4oKO8,65,"[<FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>]","[1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055]"
https://www.youtube.com/watch?v=wKOC_w4oKO8,54,"[<FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>]","[864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879]"


### Stack, Normalize, and Cast

In [75]:
@daft.func(return_dtype=dt.tensor(dt.float32(), shape=(16,288, 288, 3)))
def stack_clip(frames: list[np.ndarray], indices: list[int], clip_size: int):
    """Stacks a list of frames into a single numpy array

    Args:
        frames: List[T] of (H,W,3) float32
        indices: List[T] of int

    Returns:
        (1,T,H,W,3) float32 in [0,1]

    In a parallel/distributed groupby, a pre-group sort isn’t guaranteed
    to survive aggregation order; partitions can concatenate in
    non-deterministic order. Additionally, the image dtype is natively a
    list[uint8], so we need to cast to float32 before normalizing from
    [0,255] to [0,1].

    Steps:
    1. Aggregate both image_tensor and frame_index.
    2. Sort by frame_index inside the group-level UDF, then stack.
    3. Normalize and cast in one step.
    4. Add a batch dimension and return.

    """

    # Don't assume frames are sorted already:
    order = np.argsort(np.asarray(indices))

    # Convert Daft Image to np.ndarray
    def to_np(x):
        if hasattr(x, "to_numpy"):
            return x.to_numpy()          # Daft Image -> np.ndarray (H,W,C) uint8
        return np.asarray(x)

    # Sort frames by frame_index
    frames_sorted = [to_np(frames[i]) for i in order]

    # Ensure Tails are padded with duplicates
    if len(order) < clip_size:
        frames_sorted.extend([frames_sorted[-1]] * (clip_size - len(order)))

    # Stack, Normalize, and Cast in one step
    x = np.stack(frames_sorted[:clip_size], axis=0).astype(np.float32) / 255.0 # (T,H,W,3) float32 in [0,1]

    return x # [1,T,H,W,C] where T=clip_size

df_clips = df_grouped.with_column("clip", stack_clip(df_grouped["data"], df_grouped["frame_index"], clip_size=T))
df_clips.show(3)


pathUtf8,group_indexInt64,dataList[Image[RGB; 288 x 288]],frame_indexList[Int64],"clipFixedShapeTensor[Float32; [16, 288, 288, 3]]"
https://www.youtube.com/watch?v=wKOC_w4oKO8,97,"[<FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>]","[1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567]",<FixedShapeTensor>
https://www.youtube.com/watch?v=wKOC_w4oKO8,51,"[<FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>]","[816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831]",<FixedShapeTensor>
https://www.youtube.com/watch?v=wKOC_w4oKO8,18,"[<FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>, <FixedShapeImage>]","[288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303]",<FixedShapeTensor>


In [76]:
df_clips.count_rows()

/usr/local/lib/python3.12/dist-packages/daft/dashboard/__init__.py:91: UserWarning: Failed to broadcast metrics over http://127.0.0.1:3238/api/queries: HTTP Error 400: Bad Request
  warnings.warn(f"Failed to broadcast metrics over {url}: {e}")


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 GroupedAggregate: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Count: 00:00 

🗡️ 🐟 Project: 00:00 

128

In [92]:
B = 28
mem = 75 * 1024 * 1024 * 1024
print(mem-32212254720)

48318382080


In [93]:
@daft.udf(
    return_dtype = dt.embedding(dt.float32(), 768),
    batch_size=B, # clips per batch (tune for throughput)
    num_gpus=1,
)
class VideoPrismVideoUDF:
    def __init__(self, model_name: str = "videoprism_lvt_public_v1_base"):
        from videoprism import models as vp
        self.model = vp.get_model(model_name)
        self.params = vp.load_pretrained_weights(model_name)
        self.text_tokenizer = vp.load_text_tokenizer('c4_en')

        @jax.jit
        def vf_b(x):  # [B,T,288,288,3] -> [B,D]
            v, _, _ = self.model.apply(self.params, x, None, None, train=False)
            return v

        @jax.jit
        def vf_1(x):  # [1,T,288,288,3] -> [1,D]
            v, _, _ = self.model.apply(self.params, x, None, None, train=False)
            return v

        self.vf_b = vf_b
        self.vf_1 = vf_1

        # Warmup both
        _ = self.vf_b(jnp.zeros((B, T, H, W, C), jnp.float32)).block_until_ready()
        _ = self.vf_1(jnp.zeros((1, T, H, W, C), jnp.float32)).block_until_ready()

    def __call__(self,
        clips: list[np.ndarray], # List[T,H,W,C] of len B
    ):
        # Batch Inference
        xb = jnp.stack(clips, axis=0) # [B,T,H,W,C]
        embeddings = self.vf_b(xb) # [B,768]
        np_embeddings = np.asarray(embeddings)  # Back to NumPy
        return [np_embeddings[i].tolist() for i in range(B)]



In [94]:
df_clips_few = df_clips.sort("group_index").limit(B).collect()
df_video_embs = df_clips_few.with_column("video_embeddings", VideoPrismVideoUDF(df_clips_few["clip"])).collect()


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 GroupedAggregate: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 UDF stack_clip: 00:00 

🗡️ 🐟 TopN 28: 00:00 

🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 UDF VideoPrismVideoUDF: 00:00 

In [85]:
df_video_embs.select("group_index","video_embeddings", "clip").count_rows()

/usr/local/lib/python3.12/dist-packages/daft/dashboard/__init__.py:91: UserWarning: Failed to broadcast metrics over http://127.0.0.1:3238/api/queries: HTTP Error 400: Bad Request
  warnings.warn(f"Failed to broadcast metrics over {url}: {e}")


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Count: 00:00 

🗡️ 🐟 Project: 00:00 

16

In [ ]:
# Check GPU memory usage
!nvidia-smi

In [96]:
# Estimate JAX model memory usage
import jax
import jax.numpy as jnp
from jax.tree_util import tree_leaves

# Assuming 'self.params' holds the model parameters in your UDF
# You can adapt this to where your model parameters are stored
# For example, if your model parameters are in a variable named 'params'
# params = vp.load_pretrained_weights(MODEL_NAME)
params = vp.load_pretrained_weights("videoprism_lvt_public_v1_base")
# Get the size of each parameter in bytes
param_sizes = [x.size * x.dtype.itemsize for x in tree_leaves(params)]

# Sum up the sizes to get the total memory usage in bytes
total_param_size_bytes = sum(param_sizes)

# Convert to megabytes and gigabytes for readability
total_param_size_mb = total_param_size_bytes / (1024**2)
total_param_size_gb = total_param_size_bytes / (1024**3)

print(f"Estimated model parameter memory usage: {total_param_size_mb:.2f} MB ({total_param_size_gb:.2f} GB)")

# Note: This is an estimate of the model parameters' memory usage.
# Actual GPU memory usage during inference will also include:
# - Model activations (intermediate values computed during the forward pass)
# - Optimizer states (if training)
# - Data being processed
# - Other overhead

Estimated model parameter memory usage: 944.61 MB (0.92 GB)
